In [ ]:
!pip install TTS

In [ ]:
from IPython.display import Audio
from TTS.api import TTS
from datasets import load_dataset
import os
import gc

In [ ]:
os.mkdir("/kaggle/working/speech")

In [ ]:
%%writefile imp_term.txt
\frac
\cdot
\times
\neq
\sqrt
^
_
\alpha
\beta
\gamma
\delta
\epsilon
\theta
\lambda
\leq
\geq
\lt
\gt
\eq
\int
\sum
\prod
\lim
\int
\notin
\dots

In [ ]:
with open('imp_term.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines()]

In [ ]:
ds = load_dataset("Kyudan/MathBridge", cache_dir="/kaggle/working/temp")
ds_train = ds["train"]

In [ ]:
ds_train = ds_train.filter(lambda x: len(x["equation"])>10 and any(word in x['equation'] for word in lines) and len(x["spoken_English"])>10)
ds_train

In [ ]:
ds_train.save_to_disk("/kaggle/working/mathbridge_filtered")

In [ ]:
model_name = "tts_models/en/ljspeech/fast_pitch"
tts = TTS(model_name)

In [ ]:
# Define a function to map the text to a file path
def map_to_speech_file(example, idx):
    file_path = f"/kaggle/working/speech/tts_{idx}.mp3"
    tts.tts_to_file(text=example["spoken_English"], file_path=file_path)
    example["speech_file"] = file_path
    gc.collect()
    return example

# Apply the function to the dataset
ds_speech = ds_train.select(range(500)).map(map_to_speech_file, with_indices=True)

In [ ]:
ds_speech.save_to_disk("/kaggle/working/mathbridge_speech")